Still need to figure out:
* Missing value handing
* Running mean
* Daily climatology (logged an question in he xray google group)

In [9]:
import numpy
import pandas
import xray; print(xray.__version__)

0.5.0


### Reading data

In [2]:
infile = '/Users/damienirving/Downloads/Data/va_ERAInterim_500hPa_2006-030day-runmean_native.nc'
dset = xray.open_dataset(infile)

In [3]:
dset

<xray.Dataset>
Dimensions:    (latitude: 241, longitude: 480, time: 365)
Coordinates:
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 ...
  * latitude   (latitude) float32 -90.0 -89.25 -88.5 -87.75 -87.0 -86.25 ...
  * time       (time) datetime64[ns] 2006-01-01T06:00:00 2006-01-02T06:00:00 ...
Data variables:
    va         (time, latitude, longitude) float64 -3.818 -3.803 -3.787 ...
Attributes:
    CDI: Climate Data Interface version 1.5.3 (http://code.zmaw.de/projects/cdi)
    Conventions: CF-1.0
    history: Mon Jun 15 13:04:23 2015: ncatted -O -a axis,time,c,c,T va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Mon Jun 15 13:04:23 2015: ncatted -O -a missing_value,va,o,f,-32767. va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Mon Jun 15 13:03:30 2015: cdo seldate,2006-01-01,2006-12-31 va_ERAInterim_500hPa_030day-runmean_native.nc va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Fri Mar 20 07:35:31 2015: ncatted -O -a axis,time,c,c,T /mnt/meteo0/dat

In [4]:
va_data = dset[['va']]
type(va_data)

xray.core.dataset.Dataset

In [5]:
va_data

<xray.Dataset>
Dimensions:    (latitude: 241, longitude: 480, time: 365)
Coordinates:
  * latitude   (latitude) float32 -90.0 -89.25 -88.5 -87.75 -87.0 -86.25 ...
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 ...
  * time       (time) datetime64[ns] 2006-01-01T06:00:00 2006-01-02T06:00:00 ...
Data variables:
    va         (time, latitude, longitude) float64 -3.818 -3.803 -3.787 ...
Attributes:
    CDI: Climate Data Interface version 1.5.3 (http://code.zmaw.de/projects/cdi)
    Conventions: CF-1.0
    history: Mon Jun 15 13:04:23 2015: ncatted -O -a axis,time,c,c,T va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Mon Jun 15 13:04:23 2015: ncatted -O -a missing_value,va,o,f,-32767. va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Mon Jun 15 13:03:30 2015: cdo seldate,2006-01-01,2006-12-31 va_ERAInterim_500hPa_030day-runmean_native.nc va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Fri Mar 20 07:35:31 2015: ncatted -O -a axis,time,c,c,T /mnt/meteo0/dat

In [6]:
dset['va'].attrs

OrderedDict([(u'standard_name', u'northward_wind'), (u'long_name', u'northward_wind'), (u'units', u'm s**-1'), (u'level', u'500hPa')])

#### Missing values

[This page](http://xray.readthedocs.org/en/stable/io.html) kind of suggests that xray recognises the `_FillValue` attribute, however I haven't tried this on any data that actually has missing values (TRY ME: I need to try on SST data).

### Climatologies

You can group by `month`, `day`, `dayofyear`, `dayofweek` and whatever else belongs to the [`pandas.DatetimeIndex` class](http://pandas.pydata.org/pandas-docs/dev/generated/pandas.DatetimeIndex.html).

##### Monthly

In [7]:
mon_clim_va = va_data.groupby('time.month').mean('time')

In [8]:
mon_clim_va

<xray.Dataset>
Dimensions:    (latitude: 241, longitude: 480, month: 12)
Coordinates:
  * latitude   (latitude) float32 -90.0 -89.25 -88.5 -87.75 -87.0 -86.25 ...
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 ...
  * month      (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    va         (month, latitude, longitude) float64 -3.441 -3.431 -3.419 ...

#### Daily

(Feb 28 is day 59. In a leap year, Feb 29 is day 60.)

In [12]:
dummy_dset = xray.Dataset({'foo': ('time', numpy.random.randn(4000)),
                           'time': pandas.date_range('2000-01-01', periods=4000)})
clim = dummy_dset.groupby('time.dayofyear').mean()
anom = dummy_dset.groupby('time.dayofyear') - clim

In [13]:
dummy_dset

<xray.Dataset>
Dimensions:  (time: 4000)
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 ...
Data variables:
    foo      (time) float64 -1.197 0.8607 -0.5471 1.35 -1.297 1.012 -2.045 ...

In [18]:
clim

<xray.Dataset>
Dimensions:    (dayofyear: 366)
Coordinates:
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
Data variables:
    foo        (dayofyear) float64 -0.06518 -0.3452 0.1441 0.1311 0.06994 ...

In [36]:
test = dummy_dset.sel(time='2001-03-01')['foo'].values

numpy.ndarray

The most obvious thing to do would be:

In [58]:
day_clim_va = va_data.groupby('time.dayofyear').mean('time')
day_clim_va

<xray.Dataset>
Dimensions:    (dayofyear: 365, latitude: 241, longitude: 480)
Coordinates:
  * latitude   (latitude) float32 -90.0 -89.25 -88.5 -87.75 -87.0 -86.25 ...
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 ...
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
Data variables:
    va         (dayofyear, latitude, longitude) float64 -3.818 -3.803 -3.787 ...

However this is problematic when leap years are involved. Instead, but would be better to group by month and day, as [this](http://stackoverflow.com/questions/20973527/compute-daily-climatology-using-pandas-python) stack overflow questions suggests.

In [59]:
day_clim_va = va_data.groupby(['time.month', 'time.day']).mean('time')

AttributeError: 'list' object has no attribute 'ndim'

### Selection

It's possible to select a single dimension value or a slice of values:

In [60]:
va_data.sel(time=('2006-03-25'))

<xray.Dataset>
Dimensions:    (latitude: 241, longitude: 480, time: 1)
Coordinates:
  * latitude   (latitude) float32 -90.0 -89.25 -88.5 -87.75 -87.0 -86.25 ...
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 ...
  * time       (time) datetime64[ns] 2006-03-25T06:00:00
Data variables:
    va         (time, latitude, longitude) float64 -0.3191 -0.2874 -0.2556 ...
Attributes:
    CDI: Climate Data Interface version 1.5.3 (http://code.zmaw.de/projects/cdi)
    Conventions: CF-1.0
    history: Mon Jun 15 13:04:23 2015: ncatted -O -a axis,time,c,c,T va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Mon Jun 15 13:04:23 2015: ncatted -O -a missing_value,va,o,f,-32767. va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Mon Jun 15 13:03:30 2015: cdo seldate,2006-01-01,2006-12-31 va_ERAInterim_500hPa_030day-runmean_native.nc va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Fri Mar 20 07:35:31 2015: ncatted -O -a axis,time,c,c,T /mnt/meteo0/data/simmonds/dbirving/ERA

In [61]:
va_data.sel(time=slice('2006-03-25', '2006-04-25'))

<xray.Dataset>
Dimensions:    (latitude: 241, longitude: 480, time: 32)
Coordinates:
  * latitude   (latitude) float32 -90.0 -89.25 -88.5 -87.75 -87.0 -86.25 ...
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 ...
  * time       (time) datetime64[ns] 2006-03-25T06:00:00 2006-03-26T06:00:00 ...
Data variables:
    va         (time, latitude, longitude) float64 -0.3191 -0.2874 -0.2556 ...
Attributes:
    CDI: Climate Data Interface version 1.5.3 (http://code.zmaw.de/projects/cdi)
    Conventions: CF-1.0
    history: Mon Jun 15 13:04:23 2015: ncatted -O -a axis,time,c,c,T va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Mon Jun 15 13:04:23 2015: ncatted -O -a missing_value,va,o,f,-32767. va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Mon Jun 15 13:03:30 2015: cdo seldate,2006-01-01,2006-12-31 va_ERAInterim_500hPa_030day-runmean_native.nc va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Fri Mar 20 07:35:31 2015: ncatted -O -a axis,time,c,c,T /mnt/meteo0/d

For an arbitrary list of dimension values, you need to provide the exact date/time value (for single dates xray/pandas can guess what you meant).

In [62]:
va_data['latitude'][0:20]

<xray.DataArray 'latitude' (latitude: 20)>
array([-90.  , -89.25, -88.5 , -87.75, -87.  , -86.25, -85.5 , -84.75,
       -84.  , -83.25, -82.5 , -81.75, -81.  , -80.25, -79.5 , -78.75,
       -78.  , -77.25, -76.5 , -75.75], dtype=float32)
Coordinates:
  * latitude  (latitude) float32 -90.0 -89.25 -88.5 -87.75 -87.0 -86.25 ...
Attributes:
    standard_name: latitude
    long_name: latitude
    units: degrees_north
    axis: Y

In [63]:
va_data.sel(latitude=[-89.25, -81])

<xray.Dataset>
Dimensions:    (latitude: 2, longitude: 480, time: 365)
Coordinates:
  * latitude   (latitude) float32 -89.25 -81.0
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 ...
  * time       (time) datetime64[ns] 2006-01-01T06:00:00 2006-01-02T06:00:00 ...
Data variables:
    va         (time, latitude, longitude) float64 -4.273 -4.262 -4.251 ...
Attributes:
    CDI: Climate Data Interface version 1.5.3 (http://code.zmaw.de/projects/cdi)
    Conventions: CF-1.0
    history: Mon Jun 15 13:04:23 2015: ncatted -O -a axis,time,c,c,T va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Mon Jun 15 13:04:23 2015: ncatted -O -a missing_value,va,o,f,-32767. va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Mon Jun 15 13:03:30 2015: cdo seldate,2006-01-01,2006-12-31 va_ERAInterim_500hPa_030day-runmean_native.nc va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Fri Mar 20 07:35:31 2015: ncatted -O -a axis,time,c,c,T /mnt/meteo0/data/simmonds/dbirving/ERAInterim/d

In [64]:
va_data['time'][0:10]

<xray.DataArray 'time' (time: 10)>
array(['2006-01-01T17:00:00.000000000+1100',
       '2006-01-02T17:00:00.000000000+1100',
       '2006-01-03T17:00:00.000000000+1100',
       '2006-01-04T17:00:00.000000000+1100',
       '2006-01-05T17:00:00.000000000+1100',
       '2006-01-06T17:00:00.000000000+1100',
       '2006-01-07T17:00:00.000000000+1100',
       '2006-01-08T17:00:00.000000000+1100',
       '2006-01-09T17:00:00.000000000+1100',
       '2006-01-10T17:00:00.000000000+1100'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2006-01-01T06:00:00 2006-01-02T06:00:00 ...
Attributes:
    standard_name: time
    axis: T

In [65]:
va_data.sel(time=[numpy.datetime64('2006-01-03T17:00:00.000000000+1100'),
                  numpy.datetime64('2006-01-12T17:00:00.000000000+1100')])

<xray.Dataset>
Dimensions:    (latitude: 241, longitude: 480, time: 2)
Coordinates:
  * latitude   (latitude) float32 -90.0 -89.25 -88.5 -87.75 -87.0 -86.25 ...
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 ...
  * time       (time) datetime64[ns] 2006-01-03T06:00:00 2006-01-12T06:00:00
Data variables:
    va         (time, latitude, longitude) float64 -4.106 -4.095 -4.084 ...
Attributes:
    CDI: Climate Data Interface version 1.5.3 (http://code.zmaw.de/projects/cdi)
    Conventions: CF-1.0
    history: Mon Jun 15 13:04:23 2015: ncatted -O -a axis,time,c,c,T va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Mon Jun 15 13:04:23 2015: ncatted -O -a missing_value,va,o,f,-32767. va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Mon Jun 15 13:03:30 2015: cdo seldate,2006-01-01,2006-12-31 va_ERAInterim_500hPa_030day-runmean_native.nc va_ERAInterim_500hPa_2006-030day-runmean_native.nc
Fri Mar 20 07:35:31 2015: ncatted -O -a axis,time,c,c,T /mnt/meteo0/data/simm

#### Maths / stats

#### Basic stats along a dimension

In [66]:
outdata = va_data.mean(dim='latitude')

#### Running mean

I need to figure out how to apply the pandas [moving (rolling) statistics](http://pandas.pydata.org/pandas-docs/stable/computation.html#moving-rolling-statistics-moments) to an `xray.DataArray`. [This page](http://xray.readthedocs.org/en/stable/pandas.html) talks about how I might do that.

### Writing to file

In [67]:
outdata

<xray.Dataset>
Dimensions:    (longitude: 480, time: 365)
Coordinates:
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 ...
  * time       (time) datetime64[ns] 2006-01-01T06:00:00 2006-01-02T06:00:00 ...
Data variables:
    va         (time, longitude) float64 0.6413 0.5716 0.5214 0.4906 0.4691 ...

In [68]:
outdata.attrs = dset.attrs

In [69]:
outdata.attrs

OrderedDict([(u'CDI', u'Climate Data Interface version 1.5.3 (http://code.zmaw.de/projects/cdi)'), (u'Conventions', u'CF-1.0'), (u'history', u'Mon Jun 15 13:04:23 2015: ncatted -O -a axis,time,c,c,T va_ERAInterim_500hPa_2006-030day-runmean_native.nc\nMon Jun 15 13:04:23 2015: ncatted -O -a missing_value,va,o,f,-32767. va_ERAInterim_500hPa_2006-030day-runmean_native.nc\nMon Jun 15 13:03:30 2015: cdo seldate,2006-01-01,2006-12-31 va_ERAInterim_500hPa_030day-runmean_native.nc va_ERAInterim_500hPa_2006-030day-runmean_native.nc\nFri Mar 20 07:35:31 2015: ncatted -O -a axis,time,c,c,T /mnt/meteo0/data/simmonds/dbirving/ERAInterim/data/va_ERAInterim_500hPa_030day-runmean_native.nc\nFri Mar 20 07:34:34 2015: ncatted -O -a missing_value,va,o,f,-32767. /mnt/meteo0/data/simmonds/dbirving/ERAInterim/data/va_ERAInterim_500hPa_030day-runmean_native.nc\nFri Mar 20 07:19:21 2015: cdo runmean,30 /mnt/meteo0/data/simmonds/dbirving/ERAInterim/data/va_ERAInterim_500hPa_daily_native.nc /mnt/meteo0/data/sim

In [70]:
outdata['va'].attrs = dset['va'].attrs

In [71]:
outdata['va'].attrs

OrderedDict([(u'standard_name', u'northward_wind'), (u'long_name', u'northward_wind'), (u'units', u'm s**-1'), (u'level', u'500hPa')])

In [72]:
outdata.to_netcdf('test_xray.nc', format='NETCDF3_CLASSIC')